In [1]:
import numpy as np

def solve_qr(A, b):
    """
    Solves the square system of linear equations Ax = b using QR decomposition.

    Args:
        A (numpy.ndarray): A square n x n matrix.
        b (numpy.ndarray): A vector of length n.

    Returns:
        numpy.ndarray: The solution vector x.
        
    Raises:
        ValueError: If A is not square or dimensions mismatch.
    """
    n = A.shape[0]
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix A must be square.")
    if A.shape[0] != b.shape[0]:
        raise ValueError("Dimensions of A and b must match.")
    if len(b.shape) != 1:
         # Ensure b is a 1D vector if it was passed as a column matrix
        if b.shape[1] == 1:
            b = b.flatten()
        else:
            raise ValueError("b must be a vector.")


    # Step 1 & 2: Perform QR decomposition using numpy's library function
    Q, R = np.linalg.qr(A)

    # Step 3 & 4: Calculate y = Q^T * b
    # Q is orthogonal, so Q^-1 = Q^T
    y = np.dot(Q.T, b)

    # Step 5: Solve Rx = y using back substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):  # Loop backwards from n-1 down to 0
        sum_val = 0
        for j in range(i + 1, n):    # Sum over elements already found
            sum_val += R[i, j] * x[j]
        
        # Check for zero on diagonal (singular matrix within numerical precision)
        if np.abs(R[i, i]) < np.finfo(float).eps * 10: # Added a small tolerance check
             print(f"Warning: Potential singularity detected at R[{i},{i}] = {R[i,i]}")
             # Decide how to handle: raise error, return NaN, etc.
             # For now, proceed, but the result might be inaccurate or NaN/Inf
             # If R[i,i] is exactly zero and y[i]-sum_val is non-zero -> no solution
             # If R[i,i] is exactly zero and y[i]-sum_val is zero -> infinite solutions (can set x[i]=0 or other)

        x[i] = (y[i] - sum_val) / R[i, i]

    return x

In [2]:
import numpy as np
import math

# Define n
n = 10

# Define the matrix A
A = np.zeros((n, n))
# Note: Python uses 0-based indexing, so we adjust the formula indices
# Formula: A_ij = cos( (i-1)(j-1/2) * pi / n ) for i,j = 1..n
# In 0-based (i', j'): A_{i'j'} = cos( i' * (j' + 1/2) * pi / n ) for i', j' = 0..n-1
for i_prime in range(n):
    for j_prime in range(n):
        A[i_prime, j_prime] = math.cos(i_prime * (j_prime + 0.5) * math.pi / n)

# Define the true solution vector x_true = (1, 2, ..., n)^T
x_true = np.arange(1, n + 1)

# Define b = A * x_true
b = np.dot(A, x_true)

# --- Solve using the implemented function ---
print("Solving Ax = b using custom QR solver...")
x_computed = solve_qr(A, b)

# --- Report the results ---

# Report the computed solution x
print("\nComputed solution x:")
print(x_computed)

# Report the true solution x for comparison (optional)
# print("\nTrue solution x:")
# print(x_true)

# Calculate and report the error in the first component x1
# Note: First component corresponds to index 0 in Python
error_x1 = abs(x_computed[0] - x_true[0])
print(f"\nError in the first component (x_1): |x_computed[0] - x_true[0]| = {error_x1:.4e}")

# Optional: Calculate and report the overall error (norm)
# error_norm = np.linalg.norm(x_computed - x_true)
# print(f"\nOverall error norm ||x_computed - x_true||_2 = {error_norm:.4e}")

Solving Ax = b using custom QR solver...

Computed solution x:
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]

Error in the first component (x_1): |x_computed[0] - x_true[0]| = 8.8818e-16
